In [ ]:
import os
from PIL import Image
import numpy as np

In [ ]:
DATA_PATH = './datasets/2/'

In [ ]:
X_MEAN = np.load('x_mean.npy')

In [ ]:
def load_data(path, size_for_control):
    X = []
    y = np.concatenate([np.tile(row, (size_for_control, 1)) for row in np.eye(6)])
    for c in os.listdir(path):
        for image in os.listdir(path + '/' + c)[:size_for_control]:
            try:
                file_path = path + '/' + c + '/' + image
                img = Image.open(file_path).convert('L')
                img_data = np.asarray(img, dtype=np.int16)
                X.append(img_data)
            except:
                print('Error: ' + file_path)
    X = np.asarray(X)
    return X.reshape(len(X), *X[0].shape, 1), y

def shuffle_data(X, y):
    idx = list(range(len(X)))
    np.random.shuffle(idx)
    return X[idx], y[idx]

def preprocess(X):
    return (X.astype('float32') - X_MEAN) / 255

In [ ]:
train_size = 15000
X, y = load_data(DATA_PATH + 'train', train_size)

In [ ]:
X.shape, y.shape

In [ ]:
X_shuffled, y_shuffled = shuffle_data(X, y)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X_shuffled, y_shuffled, test_size=0.3)

In [ ]:
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

In [ ]:
#X_mean = np.mean(X_train, axis=0)

In [ ]:
X_train = preprocess(X_train)
X_valid = preprocess(X_valid)

In [ ]:
from keras.utils import to_categorical
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

In [ ]:
input_shape = X_train[0].shape

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), 
                 activation='relu', 
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (5, 5), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(6, activation='softmax'))

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy, 
              optimizer=keras.optimizers.Adadelta(), 
              metrics=['accuracy'])

In [ ]:
batch_size = 64
epochs = 25

hist = model.fit(X_train, y_train, 
          batch_size=batch_size,
          epochs=epochs, 
          verbose=1,
          validation_data=(X_valid, y_valid))

In [ ]:
score = model.evaluate(X_valid, y_valid, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
model.save('model.h5')